In [ ]:
###-----------------
### Import Libraries
###-----------------

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from collections.abc import Callable
from typing import Literal
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

%matplotlib inline

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

try:
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print (len(gpus), 'Phusical GPUs', len(logical_gpus), 'Logical GPUs')
except:
    print ('invalid device')

0 Phusical GPUs 0 Logical GPUs


In [ ]:
### Some parameters
###----------------

inpDir = '/home/dai/Documents/DNN/7.DNN/input'
outDir = '../output'
modelDir = '../models'
subDir = 'flower_photos'
altName = 'cnn_base'

RANDOM_STATE = 24 # REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results


EPOCHS = 10 # number of epochs
ALPHA = 0.001 # learning rate
TEST_SIZE = 0.2
BATCH_SIZE = 32
LR_PATIENCE = 10
FACTOR_LR = 0.5
PATIENCE = 20
IMG_HEIGHT = 190
IMG_WIDTH = 190

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 8),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = 'coolwarm' # plt.cm.Spectral

plt.rcParams.update(params)

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                  fname='flower_photos',
                                  untar=True)

data_dir = pathlib.Path(data_dir)


228813984/228813984 [==============================] - 8s 0us/step


In [ ]:
os.listdir(data_dir)

['dandelion', 'tulips', 'daisy', 'sunflowers', 'roses', 'LICENSE.txt']

In [ ]:
#create traning data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, #path to the data directory
    validation_split = TEST_SIZE, #what ratio of validation data
    subset = 'training', #purpose
    seed = RANDOM_STATE,
    image_size = [IMG_HEIGHT, IMG_WIDTH],
    batch_size = BATCH_SIZE
)

#create test data
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, #path to the data directory
    validation_split = TEST_SIZE, #what ratio of validation data
    subset = 'validation', #purpose
    seed = RANDOM_STATE,
    image_size = [IMG_HEIGHT, IMG_WIDTH],
    batch_size = BATCH_SIZE
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [ ]:
# is it picking class names
class_names = train_ds.class_names
class_names

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Build model
dor1 = 0.01
#knl_reg = tf.keras.regularizers.L2(l2 = 0)
model = tf.keras.Sequential()

# model.add(tf.keras.layers.RandomZoom(height_factor=(-0.2, -0.2), width_factor=(-0.2, -0.2)))



model.add(tf.keras.layers.Rescaling(1/255.)), # Convert between 0 and 1
model.add(tf.keras.layers.RandomFlip("horizontal"))
model.add(tf.keras.layers.RandomRotation((-0.2, 0.3)))

##-------
### Set 1
###-------
# Conv layer
model.add(tf.keras.layers.Conv2D(8, (3,3))), # 188 x 188 x 8

# model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Activation('relu')),

# model.add(tf.keras.layers.Dropout(rate=dor1, seed=RANDOM_STATE))
### Pooling
# model.add(tf.keras.layers.MaxPool2D(2, 2)), # 94 x 94 x 8

###-------
### Set 2
###-------
# Conv layer
model.add(tf.keras.layers.Conv2D(16, (3,3))), # 92 x 92 x 16

# model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Activation('relu')),

model.add(tf.keras.layers.Dropout(rate=dor1, seed=RANDOM_STATE))
### Pooling
model.add(tf.keras.layers.MaxPool2D(2, 2)), # 46 x 46 x 16

###-------
### Set 3
###-------
# Conv layer
model.add(tf.keras.layers.Conv2D(32, (3,3))), # 44 x 44 x 32

# model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Activation('relu')),

# model.add(tf.keras.layers.Dropout(rate=dor1, seed=RANDOM_STATE))
### Pooling
model.add(tf.keras.layers.MaxPool2D(2, 2)), # 22 x 22 x 32

###-------
### Set 4
###-------
# Conv layer
model.add(tf.keras.layers.Conv2D(64, (3,3))), # 20 x 20 x 64

# model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Activation('relu')),

# model.add(tf.keras.layers.Dropout(rate=dor1, seed=RANDOM_STATE))
### Pooling
model.add(tf.keras.layers.MaxPool2D(2, 2)), # 10 x 10 x 64

###-------
### Set 5
###-------
# Conv layer
model.add(tf.keras.layers.Conv2D(128, (3,3))), # 8 x 8 x 128

# model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Activation('relu')),

# model.add(tf.keras.layers.Dropout(rate=dor1, seed=RANDOM_STATE))
### Pooling
model.add(tf.keras.layers.MaxPool2D(2, 2)), # 4 x 4 x 128

###-------
### Set 6
###-------

model.add(tf.keras.layers.Conv2D(256, (3,3))), # 2 x 2 x 256

# model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Activation('relu')),

# model.add(tf.keras.layers.Dropout(rate=dor1, seed=RANDOM_STATE))
###------------
### Head Start
###------------
model.add(tf.keras.layers.Flatten()), # Flatten

model.add(tf.keras.layers.Dense(256)), # Dense 1

# model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Activation('relu')),

# model.add(tf.keras.layers.Dropout(rate=dor1, seed=RANDOM_STATE))

model.add(tf.keras.layers.Dense(5)) #Output

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    mode = 'auto',
    baseline = None,
    verbose=1,
    restore_best_weights=True,
    start_from_epoch=0,
)
lr_decay = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=FACTOR_LR,
    patience=LR_PATIENCE,
    verbose=1,
    mode='auto',

)

#define model file path
modelFile = os.path.join(modelDir, subDir, altName)

#define checkpoint callback
model_callback = tf.keras.callbacks.ModelCheckpoint(
    modelFile,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,
    mode = 'auto'
)

In [ ]:
optim = tf.keras.optimizers.AdamW(learning_rate = ALPHA)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True)

model.compile(optimizer = optim, loss = loss_fn, metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds, validation_data=test_ds,
                    epochs=EPOCHS, verbose = 1,
                    batch_size=BATCH_SIZE,
                    callbacks=[early_stopping, lr_decay, model_callback])

Epoch 1/150
72/92 [======================>.......] - ETA: 48s - loss: 1.4122 - accuracy: 0.3598

In [ ]:
model.summary()

In [ ]:
###-----------------------------------
### Function to plot Loss Curve
###-----------------------------------

def fn_plot_tf_hist(hist_df):
    '''
    Args:
      hist_df : pandas Dataframe with four columns
                For 'x' values, we will use index
    '''
    fig, axes = plt.subplots(1,2 , figsize = (15,6))

    # properties  matplotlib.patch.Patch
    props = dict(boxstyle='round', facecolor='aqua', alpha=0.4)
    facecolor = 'cyan'
    fontsize=12

    # Get columns by index to eliminate any column naming error
    y1 = hist_df.columns[0]
    y2 = hist_df.columns[1]
    y3 = hist_df.columns[2]
    y4 = hist_df.columns[3]

    # Where was min loss
    best = hist_df[hist_df[y3] == hist_df[y3].min()]

    ax = axes[0]

    hist_df.plot(y = [y1,y3], ax = ax, colormap=CMAP)


    # little beautification
    txtFmt = "{}: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(y1.capitalize(),
                           hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y3]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.95, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Min: {best[y3].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y3].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y3].to_numpy()[0]), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow

    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y1.capitalize())
    ax.set_title('Errors')
    ax.grid();
    ax.legend(loc = 'upper left') # model legend to upper left

    ax = axes[1]

    hist_df.plot( y = [y2, y4], ax = ax, colormap=CMAP)

    # little beautification
    txtFmt = "{}: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(y2.capitalize(),
                           hist_df.iloc[-1][y2],
                           hist_df.iloc[-1][y4]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.2, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Best: {best[y4].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y4].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y4].to_numpy()[0]), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow


    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y2.capitalize())
    ax.grid()
    ax.legend(loc = 'lower left')

    plt.tight_layout()

In [ ]:
loss_df = pd.DataFrame(history.history)
fn_plot_tf_hist(loss_df)